# COVID19 Search Engine

Our goal is to create a search engine that can aid people in finding information from all the papers available.

## Preprocessing
- remove stop words ("the", "and", "a", etc.)
- stemming and lemmatization (remove "-s", "-es", "-ing", "-ed") 
- Manual pattern matching --> slow, but we can use it for exploring/validation

## fast pattern matching a.
1. represent all papers as bow with words + freq counts
2. do pattern matching by looking for word in bow representations
3. Once relevant papers are found --> use context (let go of bow) to get better results and extract sentences.

## fast pattern matching b.
1. express paper, abstract, paragraphs, sentences in word embeddings. (and save somewhere)
2. express user input ass word embedding
3. comute cosine similarity of user input and paper (sentences, paragraphs, full text (whatever works best))


--> TF-IDF --> retreive pattern matches

--> pattern matching is slow. Maybe we can make a cache somehow?

--> we can use manual pattern matching as validation method


In [1]:
# unpack data
import os

import zipfile
#with zipfile.ZipFile('CORD-19-research-challenge.zip', 'r') as zip_ref:
#    zip_ref.extractall('.')

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline

import matplotlib.pyplot as plt
import spacy

import glob, json, os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.

In [4]:
root_path = '../../src/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


In [5]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

29315

In [6]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])

In [12]:
from tqdm import tqdm
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []}
for idx, entry in tqdm(enumerate(all_json), total=len(all_json)):
    if idx == 10000:
        break
    
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]

    # if only one author - or Null valie
    dict_['authors'].append(meta_data['authors'].values[0])
    
    dict_['title'].append(str(meta_data['title'])[9:-27])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal'])
df_covid.head()

 34%|████████████▌                        | 9996/29315 [01:47<03:20, 96.46it/s]

,paper_id,abstract,body_text,authors,title,journal
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...","Ward, J. C. J.; Lasecka-Dykes, L.; Neil, C.; A...",The RNA pseudoknots in foot-and-mouth disease ...,NaN
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,Hanchu Zhou; Jianan Yang; Kaichen Tang; Qingpe...,Healthcare-resource-adjusted vulnerabilities t...,NaN
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...","Butt, S. L.; Erwood, E. C.; Zhang, J.; Sellers...","Real-time, MinION-based, amplicon sequencing f...",NaN
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...,Nishi Kumari; Ayush Upadhyay; Kishan Kalia; Ra...,A Combined Evidence Approach to Prioritize Nip...,NaN
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",Shengjie Lai; Isaac Bogoch; Nick Ruktanonchai;...,Assessing spread risk of Wuhan novel coronavir...,NaN


 34%|████████████▌                        | 9996/29315 [02:00<03:20, 96.46it/s]

# Pattern matching
Print papers that contain the word transmission, incubation or covid-19.

In [14]:
import re
terms = ['transmission', 'incubation', 'covid-19']
df_covid[df_covid.str.contains(terms)]

AttributeError: 'DataFrame' object has no attribute 'str'

In [0]:
pd.options.display.max_colwidth = 300

print('papers containing ')
df_covid[(t_found[:,0]==1)][['title','authors']]

,title,authors
2,Caution: The clinical characteristics of COVID...,Zhaowei Chen; Jijia Hu; Zongwei Zhang; Shan Jiang; Tao Wang; Zhengli Shi; Zhan Zhang
5,Imaging Striatal Dopamine Release Using a Non-...,"Beyene, A. G.; Delevich, K.; Del Bonis ODonnell, J. T.; Piekarski, D. J.; Lin, W. C.; Thomas, A. W.; Yang, S. J.; Kosillo, P.; Yang, D.; Wilbrecht, L.; Landry, M. P."
6,Quantifying transmission of emerging zoonoses:...,"Ambrose, M.; Kucharski, A. J.; Formenty, P.; Muyembe-Tamfum, J.-J.; Rimoin, A. W.; Lloyd-Smith, J. O."
9,Risk estimation and prediction by modeling the...,Hui Wan; Jing-an Cui; Guo-Jing Yang
10,DEN-IM: Dengue Virus identification from shotg...,"Mendes, C. I.; Lizarazo, E.; Machado, M. P.; Silva, D. N.; Tami, A.; Ramirez, M.; Couto, N.; Rossen, J. W. A.; Carrico, J. A."
...,...,...
9715,Probable Secondary Infections in Households of...,"Lau, Joseph T.F.; Lau, Mason; Kim, Jean H.; Wong, Eric; Tsui, Hi-Yi; Tsang, Thomas; Wong, Tze Wai"
9721,Assessing hospital emergency management plans:...,"Rebmann, Terri"
9753,Detecting a trend change in cross-border epide...,"Maeno, Yoshiharu"
9776,Evaluation of Ultra-Microscopic Changes and Pr...,"Ali-Saeed, Rola; Alabsi, Aied M; Ideris, Aini; Omar, Abdul Rahman; Yusoff, Khatijah; Ali, Abdul Manaf"


In [0]:
df_covid[(t_found[:,1]==1)][['title','authors']]

,title,authors
31,Transmission interval estimates suggest pre-sy...,Lauren Tindale; Michelle Coombe; Jessica E Stockdale; Emma Garlock; Wing Yin Venus Lau; Manu Saraswat; Yen-Hsiang Brian Lee; Louxin Zhang; Dongxuan Chen; Jacco Wallinga; Caroline Colijn
49,Mapping a viral phylogeny onto outbreak trees ...,Stephen P Velsko;Jonathan E Allen;
61,A Generalized Discrete Dynamic Model for Human...,"Zhang, W.; Chen, Z.; Lu, Y.; Guo, Z.; Qi, Y.; Wang, G.; Lu, J."
63,The Viral Protein Corona Directs Viral Pathoge...,"Ezzat, K.; Pernemalm, M.; Palsson, S.; Roberts, T. C.; Jarver, P.; Dondalska, A.; Bestas, B.; Sobkowiak, M. J.; Levanen, B.; Skold, M.; Thompson, E. A.; Saher, O.; Kari, O. K.; Lajunen, T.; Sverremark Ekstrom, E.; Nilsson, C.; Ishchenko, Y.; Malm, T.; Wood, M. J. A.; Power, U. F.; Masich, S.; Li..."
67,Epidemiological and clinical features of COVID...,Penghui Yang; Yibo Ding; Zhe Xu; Rui Pu; Ping Li; Jin Yan; Jiluo Liu; Fanping Meng; Lei Huang; Lei Shi; Tianjun Jiang; Enqiang Qin; Min Zhao; Dawei Zhang; Peng Zhao; Lingxiang Yu; Zhaohai Wang; Zhixian Hong; Zhaohui Xiao; Qing Xi; Dexi Zhao; Peng Yu; Caizhong Zhu; Zhu Chen; Shaogeng Zhang; Junsh...
...,...,...
8688,DGHM 2011,NaN
8781,election of staphylococcal enterotoxin B (SEB...,"Soykut, Esra Acar; Dudak, Fahriye Ceyda; Boyacı, İsmail Hakkı"
8807,lapsing subacute demyelinating encephalomyel...,"Wege, Helmut; Watanabe, Rihito; ter Meulen, Volker"
9135,High fatality rates and associated factors in ...,"Nam, Hae-Sung; Park, Jung Wan; Ki, Moran; Yeon, Mi-Yeon; Kim, Jin; Kim, Seung Woo"


In [0]:
df_covid[(t_found[:,2]==1)&(t_found[:,0]==1)][['title','authors']]

,title,authors
2,Caution: The clinical characteristics of COVID...,Zhaowei Chen; Jijia Hu; Zongwei Zhang; Shan Jiang; Tao Wang; Zhengli Shi; Zhan Zhang
9,Risk estimation and prediction by modeling the...,Hui Wan; Jing-an Cui; Guo-Jing Yang
22,Prediction of the COVID-19 outbreak based on a...,Yuan Zhang; Chong You; Zhenghao Cai; Jiarui Sun; Wenjie Hu; Xiao-Hua Zhou
31,Transmission interval estimates suggest pre-sy...,Lauren Tindale; Michelle Coombe; Jessica E Stockdale; Emma Garlock; Wing Yin Venus Lau; Manu Saraswat; Yen-Hsiang Brian Lee; Louxin Zhang; Dongxuan Chen; Jacco Wallinga; Caroline Colijn
70,A spatial model of CoVID-19 transmission in En...,Leon Danon; Ellen Brooks-Pollock; Mick Bailey; Matt J Keeling
...,...,...
7067,écanismes d’émergence virale et transmission ...,"Gessain, Antoine"
7686,Can we contain the COVID-19 outbreak with the ...,"Wilder-Smith, Annelies; Chiew, Calvin J; Lee, Vernon J"
7725,Inactivation photochimique des pathogènes des ...,"Cazenave, J.-P."
9301,Early dynamics of transmission and control of ...,"Kucharski, Adam J; Russell, Timothy W; Diamond, Charlie; Liu, Yang; Edmunds, John; Funk, Sebastian; Eggo, Rosalind M; Sun, Fiona; Jit, Mark; Munday, James D; Davies, Nicholas; Gimma, Amy; van Zandvoort, Kevin; Gibbs, Hamish; Hellewell, Joel; Jarvis, Christopher I; Clifford, Sam; Quilty, Billy J;..."


# Word Embedding
Compute cosine similarity between questions and paper abstracts

In [0]:
nlp = spacy.load('en_core_web_lg')

OSError: ignored

In [0]:
with nlp.disable_pipes():
    vectors = np.array([nlp(text).vector for text in df_covid['abstract']])
v_mean = np.mean(vectors,axis=0)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
q1 = 'What is known about transmission, incubation, and environmental stability?'

'Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.'
'Prevalence of asymptomatic shedding and transmission (e.g., particularly children).''
'Seasonality of transmission.''
'Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).'
'Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).'
'Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic).'
'Natural history of the virus and shedding of it from an infected person'
'Implementation of diagnostics and products to improve clinical processes'
'Disease models, including animal models for infection, disease and transmission'
'Tools and studies to monitor phenotypic change and potential adaptation of the virus'
'Immune response and immunity'
'Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings'
'Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings'
'Role of the environment in transmission'

In [0]:
qv = nlp(q1).vector
cs = cosine_similarity(vectors-v_mean,qv.reshape(1,-1)-v_mean)
cs.argmax()

In [0]:
df_covid['abstract'][cs.argmax()]